In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns=None
pd.options.mode.chained_assignment = None

In [2]:
triggers_actions = pd.read_csv(r'/home/kusapochka/apr_test/triggers_actions.csv')

In [3]:
# 1. Оставляем только нужные столбцы для анализа
df = triggers_actions[['guid', 'date', 'trigger', 'type', 'result']].copy()

In [4]:
# 2. Обработка пропусков в 'result'
df['result'] = df['result'].fillna(-1)

In [5]:
# 3. Преобразуем триггеры в числовые категории
df['trigger_cat'], unique_triggers = pd.factorize(df['trigger'])

In [6]:
# 4. Сортируем по пользователям и времени взаимодействия
df= df.sort_values(by=['guid', 'date']).reset_index(drop=True)

In [7]:

df.head()


,guid,date,trigger,type,result,trigger_cat
0,0187808e-d664-cc1d-403d-b93ac1365ee8,2024-05-20 15:30:28,59,1,-1.0,305
1,0187808e-d664-cc1d-403d-b93ac1365ee8,2024-05-23 07:04:59,59,1,-1.0,305
2,0187808e-e508-2486-7385-30069e7c0a7f,2024-05-21 18:45:45,12,1,-1.0,94
3,0187808e-e508-2486-7385-30069e7c0a7f,2024-05-21 18:45:46,27,1,-1.0,110
4,0187808e-e508-2486-7385-30069e7c0a7f,2024-05-21 18:45:46,168,1,-1.0,1


In [8]:
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by=['guid', 'date'])

In [9]:
# Функция для получения всех триггеров до активного действия
def get_previous_triggers(user_triggers):
    previous_triggers = []
    for i in range(len(user_triggers)):
        # Получаем триггеры до текущего действия
        triggers_before_action = user_triggers.iloc[:i]['trigger_cat'].tolist()
        previous_triggers.append(triggers_before_action)
    return previous_triggers


In [ ]:

# Применяем функцию к каждому пользователю
df['previous_triggers'] = df.groupby('guid').apply(get_previous_triggers).reset_index(drop=True)


In [ ]:

# Преобразуем списки триггеров в новые признаки (например, мы можем использовать количество триггеров и их сумму)
df['num_previous_triggers'] = df['previous_triggers'].apply(len)
df['sum_previous_triggers'] = df['previous_triggers'].apply(lambda x: sum(x) if x else 0)

# Удаляем временные столбцы, если они не нужны
df = df.drop(columns=['previous_triggers'])

print(df[['guid', 'date', 'trigger', 'type', 'result', 'num_previous_triggers', 'sum_previous_triggers']].head())
